In [1]:
from IPython.core.magic import register_cell_magic


@register_cell_magic
def typecheck(line, cell):
    from IPython import get_ipython
    from mypy import api

    cell = "\n" + cell
    mypy_result = api.run(["-c", cell] + line.split())

    if mypy_result[0]:  # print mypy stdout
        print(mypy_result[0])

    if mypy_result[1]:  # print mypy stderr
        print(mypy_result[1])

    shell = get_ipython()
    shell.run_cell(cell)

In [2]:
import dataclasses

# Data Classesをつかってみよう
---
PyLadies Tokyo - 4周年記念パーティ LT  
2018/10/01 @コイニ―株式会社 

driller@patraqushe

### だれ？

* なまえ: どりらん
* 本名: driller

### しごと

* 本業: デリバティブのトレーダー
* 副業: 先生

### finpy

* Python x 金融のコミュニティ
* 今年で2周年  
  PyLadies Tokyoの2歳年下
* 10/27に[もくもく会](https://fin-py.connpass.com/event/102692/)やります

<img src="https://github.com/fin-py/logo/blob/master/finpy_200x200.png?raw=true" width=100 hight=100>

## こんかいのおだい

Pythonの新しい書き方をまなぶ

### 人類にはまだ早い？
* 早めに使うことで当事者意識が高まる  
  生まれたときから知っている -> 記憶に残りやすい  
  あとで知る -> アレなんだったっけ？
* 技術の変化に耐性ができる  
  古い技術に固執しない
* ドヤれる

### 新しい書き方を知らない場合
`os.path`を使ってすべての拡張子を取得する処理

In [3]:
import os


def get_suffix(filepath):
    while True:
        filepath, suffix = os.path.splitext(filepath)
        if suffix:
            yield suffix
        else:
            break


suffixes = [x for x in get_suffix("dir/filename.tar.gz")][::-1]
print(suffixes)

['.tar', '.gz']


### 新しい書き方を知ってる場合
`pathlib`使おうぜ

In [4]:
import pathlib

pathlib.Path("dir/filename.tar.gz").suffixes

['.tar', '.gz']

## Data Classes

### Data Classes

* PEP557 で提案され、Python3.7で実装
* \_\_init\_\_() や \_\_repr\_\_() のような特殊メソッドを自動で追加
* メンバ変数は型アノテーションで定義できる

### namedtupleとの違い
* 属性をミュータブル(デフォルト)/イミュータブルにできる
* 通常のクラスとして扱える  
  namedtupleは継承が必用
* \_\_init\_\_の処理をカスタマイズできる

### 従来の書き方
#### Data Classesがないとき

In [5]:
class Card:
    def __init__(self, rank: int, suit: str) -> None:
        self.rank = rank
        self.suit = suit

`__init__` の引数と `self.attr` の代入が冗長

#### Data Classesがあるとき

In [6]:
@dataclasses.dataclass
class Card37:
    rank: int
    suit: str

すっきり！

### mypyで型チェック

In [7]:
%%typecheck
# https://gist.github.com/knowsuchagency/f7b2203dd613756a45f816d6809f01a6
import dataclasses


@dataclasses.dataclass
class Card37:
    rank: int
    suit: str


Card37(1, "♥")
Card37("A", "♦")

<string>:13: error: Argument 1 to "Card37" has incompatible type "str"; expected "int"



Card37(rank='A', suit='♦')

### \_\_repr\_\_の実装
#### Data Classesがないとき

In [8]:
Card(5, "♠")

#### Data Classesがあるとき

In [9]:
Card37(5, "♠")

Card37(rank=5, suit='♠')

#### 従来のクラスで同じ機能を実装

In [10]:
class Card:
    def __init__(self, rank: int, suit: str) -> None:
        self.rank = rank
        self.suit = suit

    def __repr__(self):
        return f"{__class__.__name__}(rank={self.rank!r}, suit={self.suit!r})"


Card(5, "♠")

Card(rank=5, suit='♠')

めんどくさい

### order
#### Data Classesがあるとき

In [11]:
@dataclasses.dataclass(order=True)
class Card37:
    rank: int
    suit: str


c1 = Card37(11, "♣")
c2 = Card37(4, "♥")

print(c1 < c2)
sorted([c1, c2])

False


[Card37(rank=4, suit='♥'), Card37(rank=11, suit='♣')]

#### Data Classesがないとき

In [12]:
class Card:
    def __init__(self, rank: int, suit: str) -> None:
        self.rank = rank
        self.suit = suit

    def __repr__(self):
        return f"{__class__.__name__}(rank={self.rank!r}, suit={self.suit!r})"

    def __lt__(self, other):
        return (self.rank, self.suit) < (other.rank, other.suit)


c1 = Card(11, "♣")
c2 = Card(4, "♥")

print(c1 < c2)
sorted([c1, c2])

False


[Card(rank=4, suit='♥'), Card(rank=11, suit='♣')]

### 継承

In [13]:
@dataclasses.dataclass
class Daifugo(Card37):
    def change_rank(self, rank):
        if rank < 3:
            return rank + 13
        else:
            return rank

    def __lt__(self, other):
        return (self.change_rank(self.rank), self.suit) < (
            self.change_rank(other.rank),
            other.suit,
        )


sorted([Daifugo(2, "♦"), Daifugo(13, "♠"), Daifugo(1, "♣")])

[Daifugo(rank=13, suit='♠'),
 Daifugo(rank=1, suit='♣'),
 Daifugo(rank=2, suit='♦')]

### イミュータブル

In [14]:
@dataclasses.dataclass(frozen=True)
class Tax:
    rate: float = 0.08

tax = Tax()

In [16]:
tax.rate = 0.1

FrozenInstanceError: cannot assign to field 'rate'

### 辞書のキーに使える

In [15]:
{tax: "japan"}

{Tax(rate=0.08): 'japan'}

### さいごに

本LTは __LT駆動学習__ です  

1. いずれは使ってみたい、学んでみたい技術がある
2. 発表できそうなイベントがある
3. 先に発表することを決めてから学習する

×: いつかやってみよう  
◯: やるキッカケを先に作ってしまう

### TBD
* init
  - \_\_post_init\_\_
  - dataclasses.InitVar
* ユーティリティ関数  
  - dataclasses.fields
  - dataclasses.asdict